<a href="https://colab.research.google.com/github/henryHyun/KNU/blob/main/%EC%A4%91%EA%B0%84%EA%B3%A0%EC%82%AC_%EB%8C%80%EC%B2%B4_%EA%B3%BC%EC%A0%9C_%ED%98%84%EB%8F%99%ED%98%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 중간고사 대체 과제 (Due: 10월 29일 금요일 오후 11시 59분)

아래 코드는 statsmodel 패키지 내 Star98 데이터베이스를 다운로드 받아 이를 각각 df_festures, df_target DataFrame 객체로 저장하는 코드입니다. 

In [1]:
import sympy as sp
import numpy as np
import matplotlib.pyplot as plt
sp.init_printing(use_latex='mathjax')
import copy
import time

import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.graphics.api as smg
import patsy
import pandas as pd
import seaborn as sns
from scipy import stats

df_features = sm.datasets.star98.load_pandas().exog
df_target   = sm.datasets.star98.load_pandas().endog

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [2]:
df_features

,LOWINC,PERASIAN,PERBLACK,PERHISP,PERMINTE,AVYRSEXP,AVSALK,PERSPENK,PTRATIO,PCTAF,PCTCHRT,PCTYRRND,PERMINTE_AVYRSEXP,PERMINTE_AVSAL,AVYRSEXP_AVSAL,PERSPEN_PTRATIO,PERSPEN_PCTAF,PTRATIO_PCTAF,PERMINTE_AVYRSEXP_AVSAL,PERSPEN_PTRATIO_PCTAF
0,34.39730,23.299300,14.235280,11.411120,15.918370,14.70646,59.15732,4.445207,21.71025,57.03276,0.0,22.222220,234.102872,941.68811,869.9948,96.50656,253.52242,1238.1955,13848.8985,5504.0352
1,17.36507,29.328380,8.234897,9.314884,13.636360,16.08324,59.50397,5.267598,20.44278,64.62264,0.0,0.000000,219.316851,811.41756,957.0166,107.68435,340.40609,1321.0664,13050.2233,6958.8468
2,32.64324,9.226386,42.406310,13.543720,28.834360,14.59559,60.56992,5.482922,18.95419,53.94191,0.0,0.000000,420.854496,1746.49488,884.0537,103.92435,295.75929,1022.4252,25491.1232,5605.8777
3,11.90953,13.883090,3.796973,11.443110,11.111110,14.38939,58.33411,4.165093,21.63539,49.06103,0.0,7.142857,159.882095,648.15671,839.3923,90.11341,204.34375,1061.4545,9326.5797,4421.0568
4,36.88889,12.187500,76.875000,7.604167,43.589740,13.90568,63.15364,4.324902,18.77984,52.38095,0.0,0.000000,606.144976,2752.85075,878.1943,81.22097,226.54248,983.7059,38280.2616,4254.4314
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
298,62.23968,13.820860,3.956157,32.128790,15.884480,13.27089,52.88133,4.324174,20.98711,8.00000,0.0,18.181820,210.801187,839.99243,701.7823,90.75192,34.59339,167.8969,11147.4471,726.0153
299,46.16188,1.150575,0.800400,45.622810,13.333330,11.80714,48.42615,3.945069,19.85830,53.77358,0.0,0.000000,157.428494,645.68184,571.7743,78.34236,212.14048,1067.8519,7623.6559,4212.7494
300,43.41431,3.648564,1.541204,46.319980,21.377670,14.40468,53.79616,3.845481,22.70884,29.89247,0.0,6.666667,307.938495,1150.03656,774.9165,87.32641,114.95093,678.8233,16565.9086,2610.4022
301,70.62104,20.461720,2.989345,17.423050,11.032860,16.56420,61.70566,3.970984,24.88647,15.14286,0.0,13.636360,182.750500,680.78991,1022.1049,98.82377,60.13205,376.8523,11276.7402,1496.4746


In [3]:
df_target

,NABOVE,NBELOW
0,452.0,355.0
1,144.0,40.0
2,337.0,234.0
3,395.0,178.0
4,8.0,57.0
...,...,...
298,96.0,246.0
299,37.0,117.0
300,208.0,387.0
301,199.0,510.0


위 데이터는 1998년 미국 캘리포니아 교육부에서 초중고등학생들을 대상으로 (2nd ~ 11th grade) 실시한 지역(county)별 수학 과목 학력평가 시험 결과 및 각 지역 내 학업 성적과 관련이 있을 수 있는 변수들을 조사한 결과를 담고 있습니다. 각 feature들에 대한 설명은 다음과 같습니다. (자세한 설명은 [여기를](https://www.statsmodels.org/stable/datasets/generated/star98.html) 참조)

*   df_target 내에서:
>* **NABOVE**: 해당 지역 내 전국 성적 중앙값을 넘은 점수를 받은 학생들의 수.
>* **NBELOW**: 해당 지역 내 전국 성적 중앙값에 미치지 못하는 점수를 받은 학생들의 수 (따라서, NABOVE + NBELOW = 해당 지역 총 학생 수).

* df_features 내에서:
>* **LOWINC**: 해당 지역 내 저소득층 학생 비율.
>* **PERASIAN**: 해당 지역 내 아시아계 학생 비율.
>* **PERBLACK**: 해당 지역 내 흑인 학생 비율.
>* **PERHISP**: 해당 지역 내 히스패닉 학생 비율.
>* **PERMINTE**: 해당 지역 내 소수인종 교사 비율.
>* **AVYRSEXP**: 해당 지역 내 교사 평균 근속년수.
>* **AVSALK**: 해당 지역 내 교사 평균 연봉 (단위: 1000달러). 
>* **PERSPENK**: 해당 지역 내 학생 1명당 교육 예산 지출 (단위: 1000달러). 
>* **PTRATIO**: 해당 지역 내 교사 1명 당 학생 수.
>* **PCTAF**: 해당 지역 내 캘리포니아 주립대에서 제공하는 prep course(물리, 화학, 생물 등의 과학 과목을 대학교에서 미리 배울 수 있는 제도)를 수강하는 학생 비율. 
>* **PCTCHRT**: 해당 지역 내 차터 스쿨 (https://ko.wikipedia.org/wiki/차터_스쿨) 비율.
>* **PCTYRRND**: 해당 지역 내 year-round 스쿨 (https://www.edweek.org/leadership/year-round-schooling-explained/2015/12) 비율. 
>* **PERMINTE_AVYRSEXP** = PERMINTE * AVYRSEXP.
>* **PEMINTE_AVSAL** = PERMINTE * AVSALK. 
>* **AVYRSEXP_AVSAL** = AVYRSEXP * AVSALK. 
>* **PERSPEN_PTRATIO** = PERSPENK * PTRATIO.
>* **PERSPEN_PCTAF** = PERSPENK * PCTAF.
>* **PTRATIO_PCTAF** = PTRATIO * PCTAF. 
>* **PERMINTE_AVTRSEXP_AVSAL** = PERMINTE * AVYRSEXP * AVSALK. 

## 문제: 
위 자료들을 이용하여, 학생들의 학업 성적과 다른 변수들 사이의 관계를 설명하는 통계적 모델을 만들고, 이를 통해 학생들의 성적에 중요한 영향을 끼치는 요인들에 어떤 것들이 있는지를 통계 모델 분석 및 시각화를 근거로 서술하세요.  

In [4]:
#sns.set(style='whitegrid') # whitegrid = 배경에 하얀 선
#sns.pairplot(df_features)
#plt.show()

In [5]:
# 열 이름 바꾸기 
df_target.columns = ['Good_grade', 'Bad_grade']

In [6]:
# 각각 변수 지정 
good_df = df_target[['Good_grade']]
bad_df = df_target[['Bad_grade']]

In [7]:
# 평균값 구하기(우등성적생)
good_df.mean()

Good_grade    357.815182
dtype: float64

In [8]:
# 평균값 구하기(열등성적생)
bad_df.mean()

Bad_grade    525.389439
dtype: float64

In [9]:
# '토탈'이라는 이름의 데이터프레임 만들기 
df_target['sum'] = df_target.sum(axis=1)
df_target['sum']

0      807.0
1      184.0
2      571.0
3      573.0
4       65.0
       ...  
298    342.0
299    154.0
300    595.0
301    709.0
302    156.0
Name: sum, Length: 303, dtype: float64

In [10]:
# 행 별 학생수의 합을 나타내는 열 추가
df_target

,Good_grade,Bad_grade,sum
0,452.0,355.0,807.0
1,144.0,40.0,184.0
2,337.0,234.0,571.0
3,395.0,178.0,573.0
4,8.0,57.0,65.0
...,...,...,...
298,96.0,246.0,342.0
299,37.0,117.0,154.0
300,208.0,387.0,595.0
301,199.0,510.0,709.0


In [11]:
# 전체 학생 수 
df_target['sum'].sum()

267611.0

In [12]:
#원하는 열 분리하기
#분리한 열만 시각화
#그래프 그려서 비교하기
#상관 관계 설명하기
#변수 여러개 지정 하기
#각 변수별 평균 구하기 그리고 평균 위 아래 구분 하기
#전체 학생수 × 인종 비율 = 실제 인종별 학생 수  

In [13]:
# 원하는 열 분리하기
# 인종별 학생비율표 
data = df_features[['PERASIAN','PERBLACK','PERHISP','PERMINTE']]
data.columns = ['아시아계', '흑인계', '히스패닉계', '소수인종']

In [14]:
data = df_features[['PERASIAN']]
data.columns = ['아시아계']
data.sort_values('아시아계', ascending=False)

,아시아계
116,63.20234
85,52.59396
125,42.31389
231,41.36902
119,38.10775
...,...
64,0.00000
147,0.00000
55,0.00000
77,0.00000


In [15]:
# 아시아계 학생비율의 평균값 = 아시아계 학생들의 총합 / 아시아 학생 조사 수 
total_asia = df_features['PERASIAN'].sum()
total_asia
asia_mean = total_asia / 303
asia_mean
# 아시아계 학생들의 평균 비율은 5.89퍼센트?

5.896335402640264

In [16]:
df_target['sum'].sum() * asia_mean

1577924.2134359637

In [17]:
data = df_features[['PERBLACK']]
data.columns = ['흑인계']
data.sort_values('흑인계', ascending=False)

,흑인계
4,76.87500
10,50.94093
103,43.09138
2,42.40631
122,35.72962
...,...
57,0.00000
142,0.00000
25,0.00000
281,0.00000


In [18]:
# 흑인계 학생비율의 평균값 
df_features['PERBLACK'].mean()

5.636807570957095

In [19]:
data = df_features[['PERHISP']]
data.columns = ['히스패닉계']
data.sort_values('히스패닉계', ascending=False)

,히스패닉계
43,98.821000
49,97.769160
59,97.519980
186,96.167130
99,94.480520
...,...
56,2.988954
55,2.842566
11,2.587519
294,2.345216


In [20]:
# 히스패닉계 학생비율의 평균값 
df_features['PERHISP'].mean()

34.39807966666665

In [21]:
data = df_features[['PERMINTE']]
data.columns = ['소수인종']
data.sort_values('소수인종', ascending=False)

,소수인종
122,80.174930
59,70.099670
103,63.439070
43,62.184870
110,59.529810
...,...
135,1.459854
81,1.204819
136,0.000000
57,0.000000


In [22]:
# 소수 인종 학생비율의 평균값 
df_features['PERMINTE'].mean()

14.694746821782173

In [23]:
# 저소득층 학생비율 만 존재하는 열 생성
data_low = df_features[['LOWINC']]
data_low.columns = ['저소득층']
data_low.sort_values('저소득층', ascending=False)

,저소득층
287,92.334500
43,91.604840
46,87.837180
49,87.777250
50,86.545990
...,...
104,1.421801
294,1.247051
116,0.265340
11,0.000000


In [24]:
# 저소득층 학생들의 평균 비율
data_low.mean()

저소득층    41.409877
dtype: float64